In [48]:
from sqlalchemy import create_engine
import pandas as pd
from os import listdir
from os.path import isfile, join

In [49]:
CONFIG = {}

with open('config.txt', "r") as in_file:
    for line in in_file:
        line = line.split(":")
        parameter = line[0].strip()
        value = line[1].strip()
        CONFIG[parameter] = value
                
engine_path = 'postgresql://'+CONFIG['username']+':'+\
                CONFIG['password']+'@'+CONFIG['ip']+\
                ':'+CONFIG['port']+'/'+CONFIG['database']
        
engine = create_engine(engine_path)

### Want something like:

    SELECT
        table4.countrycode as 'countrycode',
        b.table1 as 'table1',
        b.table2 as 'table2',
        b.table3 as 'table3',
        table4.year as 'table4'
        (SELECT
            table3.countrycode as 'countrycode',
            a.table1 as 'table1',
            a.table2 as 'table2',
            table3.year as 'table3'
         FROM
            (SELECT
                 table1.countrycode as 'countrycode',
                 table1.year as 'table1',
                 table2.year as 'table2'
             FROM
                 table1
             JOIN
                 table2
               ON 
                 table1.countrycode = table2.countrycode
             ) a
         JOIN
             table3
           ON
             a.countrycode = table3.countrycode  
        ) b    
    JOIN
        table4
      ON
        a.countrycode = table3.countrycode 
    
        
       


In [50]:
def makeSQLQuery(table_names, year):
    dotyyear = '.y' + str(year)
    current_unit = 'a'
    query = '(SELECT ' + table_names[0] + '.countrycode AS countrycode, ' + \
            table_names[0] + dotyyear + ' AS ' + table_names[0] + ', ' + \
            table_names[1] + dotyyear + ' AS ' + table_names[1] + \
            ' FROM ' + table_names[0] + \
            ' JOIN ' + table_names[1] + \
            ' ON ' + table_names[0] + '.countrycode=' + table_names[1] + '.countrycode) '
    query = query + current_unit
    
    for table in table_names[2:]:
        curr_index = table_names.index(table)
        temp_list = table_names[:curr_index+1]
        
        pre_string = '(SELECT ' + table + '.countrycode as countrycode'
        
        for temp_table in temp_list[:-1]:
            pre_string = pre_string + ', ' + current_unit + '.' + temp_table + ' AS ' + temp_table
        
        pre_string = pre_string + ', ' + table + dotyyear + ' AS ' + table
        
        query = pre_string + ' FROM ' + query + ' JOIN ' + table + \
                ' ON ' + current_unit + '.countrycode=' + table + '.countrycode) '
        
        current_unit = current_unit + 'a'
        query = query + current_unit
        
    query = query.rsplit(' ', 1)[0][1:-1]
    
    return query
    
    

In [51]:
table_names = ['corruption_perception', 'life_expectancy', \
               'population_0_14', 'population_15_64', 'population_65_up', \
               'population_female', 'under_5_mortality', 'zscores']

In [52]:
query = makeSQLQuery(table_names, 2013)

In [53]:
pd.read_sql_query(query, engine)

,countrycode,corruption_perception,life_expectancy,population_0_14,population_15_64,population_65_up,population_female,under_5_mortality,zscores
0,AFG,8,60.028268,45.685257,51.920329,2.394414,48.507090,96.7,6.005489869
1,AGO,23,51.866171,48.012577,49.673340,2.314084,50.407447,167.1,-0.232733228
2,ALB,31,77.537244,19.382294,68.848024,11.769682,50.073406,14.9,-0.003833417
3,ARE,69,77.195634,13.718086,85.340053,0.941861,25.883957,7.4,-0.265035505
4,ARG,34,75.986098,25.465743,63.834426,10.699832,51.078024,13.3,-0.26509832
5,ARM,36,74.561366,19.013085,70.546141,10.440774,51.604376,15.5,0.03658775
6,AUS,81,82.197561,18.783295,66.838548,14.378157,50.004824,4.1,-0.265693695
7,AUT,69,80.890244,14.355584,67.220273,18.424143,51.036378,3.9,-0.265696061
8,AZE,28,70.693146,22.079936,72.280894,5.639171,50.283991,34.2,-0.179360882
9,BDI,21,56.251610,44.482947,53.032264,2.484789,50.621668,87.8,0.256683304
